## Legal RAG Solution

## 1. Setup

Importing Libraries

In [3]:
! pip install langchain_openai langchain whisper langchain_community scikit-learn langchain_pinecone langchain[docarray] docarray pydantic==1.10.8 pytube  python-dotenv tiktoken ruff --quiet

Loading the environment variables we need to use.

In [2]:
import os
os.environ['OPENAI_API_KEY'] = ""
os.environ['PINECONE_API_KEY'] = ""
os.environ['PINECONE_ENV'] = ""


Let's define the LLM model that we'll use as part of the workflow.

In [4]:
from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI(openai_api_key= 'OPENAI_API_KEY', model="gpt-3.5-turbo")

## 2. Loading the document

In [7]:
! pip install datasets

In [6]:
from datasets import load_dataset
dataset = load_dataset("ninadn/indian-legal",)
dataset

/home/codespace/.python/current/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['Text', 'Summary'],
        num_rows: 7030
    })
    test: Dataset({
        features: ['Text', 'Summary'],
        num_rows: 100
    })
})

In [8]:
import pandas as pd
df = pd.DataFrame(dataset['test'])
df


,Text,Summary
0,Appeal No. 101 of 1959.\nAppeal by special lea...,The appellants who are displaced persons from ...
1,Appeal No. 52 of 1957.\nAppeal from the judgme...,The appellants and the respondents were owners...
2,Appeals Nos. 45 and 46 of 1959.\nAppeal by spe...,The respondents firm claimed exemption from Sa...
3,ION: Criminal Appeal 89 of 1961.\nAppeal by sp...,The appellant was tried for murder.\nThe facts...
4,Civil Appeal No. 50 of 1961.\nAppeal by specia...,"S, employed by the appellant as a cross cutter..."
...,...,...
95,Appeal No. 1367 of 1980.\nFrom the Judgment an...,Proceedings were commenced under Chapter III B...
96,Appeal No. 1695 of 1993.\nFrom the Judgment an...,"The plaintiff, predecessor in interest of the ..."
97,iminal Appeal No. 46 of 1957.\nAppeal by speci...,Conciliation proceedings were started in Janua...
98,N: Criminal Appeal No. 8 of 1951.\nAppeal from...,Sub section (1) of sec.\n19 of the Bombay Rent...


In [9]:
df = df.drop(columns=['Summary'])


In [10]:
df

,Text
0,Appeal No. 101 of 1959.\nAppeal by special lea...
1,Appeal No. 52 of 1957.\nAppeal from the judgme...
2,Appeals Nos. 45 and 46 of 1959.\nAppeal by spe...
3,ION: Criminal Appeal 89 of 1961.\nAppeal by sp...
4,Civil Appeal No. 50 of 1961.\nAppeal by specia...
...,...
95,Appeal No. 1367 of 1980.\nFrom the Judgment an...
96,Appeal No. 1695 of 1993.\nFrom the Judgment an...
97,iminal Appeal No. 46 of 1957.\nAppeal by speci...
98,N: Criminal Appeal No. 8 of 1951.\nAppeal from...


In [11]:
df.head(18)

,Text
0,Appeal No. 101 of 1959.\nAppeal by special lea...
1,Appeal No. 52 of 1957.\nAppeal from the judgme...
2,Appeals Nos. 45 and 46 of 1959.\nAppeal by spe...
3,ION: Criminal Appeal 89 of 1961.\nAppeal by sp...
4,Civil Appeal No. 50 of 1961.\nAppeal by specia...
5,iminal Appeals Nos.\n160 to 162 of 1960.\nAppe...
6,l Appeals Nos.\n15 to 19 of 1962.\nAppeal from...
7,"Appeals, Nos. 275 276 of 1963.\nAppeals by spe..."
8,Appeals Nos. 884 887 of 1962.\nAppeals from th...
9,Appeal No. 251 of 1963.\nAppeal by special lea...


In [12]:
df.to_csv('data_space_separated.txt', sep=' ', index=False)


In [13]:
df.to_csv('data.csv', index=False, sep=';')


In [14]:
from google.colab import files
files.download('data.csv')

ModuleNotFoundError: No module named 'google'

In [15]:
from langchain_community.document_loaders.csv_loader import CSVLoader
import sys
import csv
csv.field_size_limit(sys.maxsize)
loader = CSVLoader(file_path='data.csv')
data = loader.load()

In [16]:
with open("data.csv") as file:
    transcription = file.read()

transcription[:100]

'Text\n"Appeal No. 101 of 1959.\nAppeal by special leave from the judgment and order dated November 8, '

## 3. Chunking & Indexing the CSV

In [18]:
from langchain_community.document_loaders import CSVLoader

loader = CSVLoader("data.csv")
text_documents = loader.load()
text_documents

[Document(metadata={'source': 'data.csv', 'row': 0}, page_content='Text: Appeal No. 101 of 1959.\nAppeal by special leave from the judgment and order dated November 8, 1957, of the Deputy Custodian General, Evacuee Property, Now Delhi Revision Petition No. 17 R/55 of 1955.\nAchhru Ram and K. L. Mehta for the appellants.\nB.K., Khanna and, T. M. Sen, for the respondent No. 1.\nN.S. Bindra and A. G. Ratnaparkhi, for the respondents Nos.\nMarch 15.\nThe Judgment of the Court was delivered by MUDHOLKAR J.\nThe appellants who are admittedly displaced persons from West Pakistan were granted quasi permanent allotment of 24 standard acres and 15 3/4 units in the village of Raikot in Ludhiana District in 1949.\nTheir father Sardar Nand Singh who was 42 330 found entitled to quasi permanent allotment of 40 standard acres and 5 1/4 units of land was given quasipermanent allotment in another village named Humbran in the same district.\nThe two villages are, however, 25 miles or so distant from eac

In [20]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents=text_splitter.split_documents(text_documents)

## 4. Creating embeddings

Let's generate embeddings for an arbitrary query:

In [21]:
from langchain_openai.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()


## 5. Using Pinecone as Vector Database

In [23]:
from langchain_pinecone import PineconeVectorStore

index_name = "tmf17"

vectorstore = PineconeVectorStore.from_documents(
    documents, embeddings, index_name=index_name
)

--------------------------------------------------    INGESTION COMPLETES HERE     -----------------------------------------------------------

### Retrival & Generation

In [24]:
query = ["How did the court interpret the terms annual charge & capital charge under section 9 (1) (iv) of the Indian Income Tax Act in the case of Appeal No. LXVI of 1949? "]

results = []
for q in query:
    result = vectorstore.similarity_search(q)
    results.append(result)


In [25]:
from langchain.prompts import ChatPromptTemplate

results = []
all_contexts = []

# Loop through each query, retrieve context, and store it
for q in query:
    result = vectorstore.similarity_search(q)
    context_text = ([doc.page_content for doc in result])
    all_contexts.append(context_text)

In [27]:
from langchain.prompts import ChatPromptTemplate


# Combine all contexts into a single string
#final_context = "\n\n".join(all_contexts)
#print(final_context)

# Create prompt template
PROMPT_TEMPLATE = """
Answer the question based only on the following context:
{context}
Answer the question based on the above context: {question}.
Provide a detailed answer.
Don’t justify your answers.
Don’t give information not mentioned in the CONTEXT INFORMATION.
Do not say "according to the context" or "mentioned in the context" or similar.
"""

prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)

In [28]:
from langchain.prompts import ChatPromptTemplate

responses = []
for i, q in enumerate(query):
    prompt = prompt_template.format(context=all_contexts[i], question=q)
    model = ChatOpenAI()
    response_text = model.predict(prompt)
    responses.append(response_text)


for response in responses:
    print(response)

/home/codespace/.python/current/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


The court interpreted the terms annual charge and capital charge under section 9(1)(iv) of the Indian Income Tax Act in the case of Appeal No. LXVI of 1949 by considering the specific facts and circumstances presented in the case. The court examined the applicability of these terms to the particular situation of the assessee, which was a Hindu undivided family headed by Sri Trivikram Narain Singh, a descendant of Sri Babu Ausan Singh, the original founder of Ausanganj State in the district of Benaras. The court also took into account the historical background related to the Treaty between the East India Company and Nawab Asfuddaula in 1775, which ceded the province of Benaras to the British Government. Based on these facts, the court likely analyzed whether the sum of Rs. 36,396 received by the assessee as an allowance during the previous year of the assessment year 1949-50 constituted revenue income liable to tax under the Indian Income Tax Act of 1922.


In [33]:
df

,Text
0,Appeal No. 101 of 1959.\nAppeal by special lea...
1,Appeal No. 52 of 1957.\nAppeal from the judgme...
2,Appeals Nos. 45 and 46 of 1959.\nAppeal by spe...
3,ION: Criminal Appeal 89 of 1961.\nAppeal by sp...
4,Civil Appeal No. 50 of 1961.\nAppeal by specia...
...,...
95,Appeal No. 1367 of 1980.\nFrom the Judgment an...
96,Appeal No. 1695 of 1993.\nFrom the Judgment an...
97,iminal Appeal No. 46 of 1957.\nAppeal by speci...
98,N: Criminal Appeal No. 8 of 1951.\nAppeal from...


In [37]:
import pandas as pd
df = pd.read_csv('qa.csv')

# Initialize lists for queries and responses
queries = df['question']
responses = []

In [39]:
# Loop through each query, invoke response and store in responses list
for query in queries:
    response = model.invoke(query)  # Replace with your actual chain.invoke(Query) function
    responses.append(response)

In [40]:
#df = df.drop(columns=['id'])
df['answer'] = responses
#df["contexts"]
print("Existing columns:", df.columns)



Existing columns: Index(['question', 'ground_truth', 'answer'], dtype='object')


In [42]:
pd.set_option('display.max_colwidth', None)

In [44]:
df

,question,ground_truth,answer
0,What was the main legal issue in the appeal case of Nand Singh and his sons concerning the allotment of land in the village Raikot?,The main legal issue in the appeal case was whether the Deputy Custodian General had the jurisdiction to revise the order canceling the allotment of land in Raikot made in favor of the appellants after the enactment of the 1954 Act and the notification issued thereunder.,"content=""The main legal issue in the appeal case of Nand Singh and his sons concerning the allotment of land in the village Raikot was whether the lower court had erred in finding that the plaintiffs were entitled to the land in question based on their alleged possession and inheritance rights, despite the defendant's claim that they had acquired the land through a valid purchase agreement."" additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 73, 'prompt_tokens': 37, 'total_tokens': 110}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-723bd1ad-4f0e-4854-a03c-e5aa043f9ce0-0' usage_metadata={'input_tokens': 37, 'output_tokens': 73, 'total_tokens': 110}"
1,"On what grounds did the Deputy Custodian General dismiss the appellants' application in the judgment dated November 8, 1957?","The Deputy Custodian General dismissed the appellants' application on the grounds that his jurisdiction to revise the order had been taken away by the provisions of the 1954 Act and the notification issued on March 24, 1955. The decision relied on the precedent set in the case of Bal Mukund vs. The State of Punjab.","content=""The Deputy Custodian General dismissed the appellants' application in the judgment dated November 8, 1957 on the grounds that they had failed to provide sufficient evidence to support their claim for relief. The appellants were unable to demonstrate that they were entitled to the relief sought under the relevant laws and regulations governing their case. Additionally, the Deputy Custodian General found that the appellants had not met the burden of proof required to show that they had a valid legal basis for their application."" additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 98, 'prompt_tokens': 34, 'total_tokens': 132}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-2515a3d1-f519-4f58-a192-6e2104e9010f-0' usage_metadata={'input_tokens': 34, 'output_tokens': 98, 'total_tokens': 132}"
2,What was the main legal issue concerning the timing of when property in the goods passed in the context of the sales on FOB contracts in this case?,"The main legal issue was whether the property in the goods passed to the buyers at the time of shipment (i.e., after crossing the customs frontier) or at some point before shipment. The sellers argued that the property passed on shipment, thus exempting them from sales tax under article 286(1)(b) of the Constitution. The court had to determine if the sales occurred ""in the course of export"" and if the goods were indeed exempt from sales tax based on the timing of the property transfer.","content='The main legal issue concerning the timing of when property in the goods passed in the context of the sales on FOB contracts in this case was whether the property in the goods passed to the buyer at the time the goods were loaded onto the vessel or at the time the vessel sailed. This determination is important because it affects which party bears the risk of loss or damage to the goods during transit.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 78, 'prompt_tokens': 37, 'total_tokens': 115}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-95c84ac8-65d1-42ef-b5be-ef200d46781c-0' usage_metadata={'input_tokens': 37, 'output_tokens': 78, 'total_tokens':

### Evaluation
